In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

dataset = pd.read_csv("dublinbikes_20200101_20200401.csv")
df = dataset

df1 = df.copy()
df1['STATUS'] = df1['STATUS'].astype('category')
df1['LAST UPDATED'] = pd.to_datetime(df1['LAST UPDATED'])
df1['TIME'] = pd.to_datetime(df1['TIME'])
df1['formatted_time'] = df1['TIME'].dt.floor('h')

df1['day_of_week'] = df1['formatted_time'].dt.strftime('%A')
df1['day_of_month'] = df1['formatted_time'].dt.strftime('%d').astype(np.int64)
df1['hour'] = df1['formatted_time'].dt.strftime('%H').astype(np.int64)
df1['month'] = df1['formatted_time'].dt.strftime('%m').astype(np.int64)
df1['week'] = df1['formatted_time'].dt.strftime('%w').astype(np.int64)
numeric_columns = df1.select_dtypes(['int64']).columns
print(df1[numeric_columns].describe().T)
print(df1.select_dtypes(['category']).describe().T)
print(df1.dtypes)

ts = pd.to_datetime('2020/02/01')
te = pd.to_datetime('2020/04/01')
mask = (df1['TIME'] >= ts) & (df1['TIME'] <= te)
pd.options.mode.chained_assignment = None
#df['TIME'].max() - df['TIME'].min()
#suburb_point = "Merrion Square South"
suburb_point = "Grangegorman Lower (South)"
suburb_df = df1.loc[mask]
suburb_df = suburb_df[suburb_df['ADDRESS'] == suburb_point]
suburb_dataset = suburb_df[['TIME', 'AVAILABLE BIKES']]
#fig, ax = plt.subplots()

suburb_dataset['date'] = suburb_dataset['TIME'].dt.floor('T')
suburb_dataset = suburb_dataset.reset_index()
suburb_dataset.drop('TIME',axis=1, inplace=True)
time = suburb_dataset['date']
bikes = suburb_dataset['AVAILABLE BIKES']
#plot 2 week data for Merrion Square South region
fig = plt.figure()
ax = fig.add_subplot(111)
ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.scatter(time,bikes, color="red", marker=".", label='bike count')
ax.set_xlabel("date")
ax.set_ylabel("bike available")
fig.autofmt_xdate()
ax.grid(True)
ax.set_title(f'Bike usage for {suburb_point}')
plt.show()

                           count       mean        std   min   25%   50%  \
STATION ID             2228278.0  60.196628  33.598271   2.0  31.0  61.0   
BIKE STANDS            2228278.0  32.184342   7.666891  16.0  29.0  30.0   
AVAILABLE BIKE STANDS  2228278.0  20.456110  11.140868   0.0  12.0  20.0   
AVAILABLE BIKES        2228278.0  11.644086   9.980968   0.0   3.0  10.0   
day_of_month           2228278.0  15.902037   9.262035   1.0   8.0  16.0   
hour                   2228278.0  11.556068   6.876381   0.0   6.0  12.0   
month                  2228278.0   2.317890   0.729879   1.0   2.0   2.0   
week                   2228278.0   2.991054   1.976522   0.0   1.0   3.0   

                        75%    max  
STATION ID             90.0  117.0  
BIKE STANDS            40.0   40.0  
AVAILABLE BIKE STANDS  29.0   41.0  
AVAILABLE BIKES        18.0   40.0  
day_of_month           24.0   31.0  
hour                   17.0   23.0  
month                   3.0    4.0  
week               

In [4]:
from sklearn.metrics import mean_squared_error
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import numpy
from tensorflow.keras.optimizers import Adam
import keras
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

#%matplotlib qt 

t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

day_of_month = suburb_df['day_of_month']
hour = suburb_df['hour']

q=12
lag=3
stride=1
w=math.floor(7*24*60*60/dt)
length = bikes.size - w - lag * w - q
print("intial set : ", bikes.size, w, lag, q, length)
XX=bikes[q:q+length:stride]
X1 = day_of_month[q:q+length:stride]
X2 = hour[q:q+length:stride]
XX=np.column_stack((XX,X1, X2))
#week
for i in range(1,lag):
    X=bikes[i*w+q:i*w+q+length:stride]
    X1 = day_of_month[i*w+q:i*w+q+length:stride]
    X2 = hour[i*w+q:i*w+q+length:stride]
    XX=np.column_stack((XX,X, X1, X2))
d=math.floor(24*60*60/dt)
#days
for i in range(0,lag):
    X=bikes[i*d+q:i*d+q+length:stride]
    X1=day_of_month[i*d+q:i*d+q+length:stride]
    X2=hour[i*d+q:i*d+q+length:stride]
    XX=np.column_stack((XX,X, X1, X2))

for i in range(0,lag):
    X=bikes[i:i+length:stride]
    X1=day_of_month[i:i+length:stride]
    X2=hour[i:i+length:stride]
    XX=np.column_stack((XX,X, X1, X2))

yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
tt=time[lag*w+w+q:lag*w+w+q+length:stride]

yy.reset_index(drop=True, inplace=True)
tt.reset_index(drop=True, inplace=True)

train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

model = Sequential()
model.add(Dense(128, activation="relu", input_dim=27))
model.add(Dense(32, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(1, activation="linear"))
model.summary()
model.compile(loss='mean_squared_error', optimizer=Adam(lr=1e-3, decay=1e-3 / 200))
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
# 10000000
# 15000
history = model.fit(XX[train], yy[train], validation_data=(XX[test], yy[test]), epochs=5000, batch_size=100, verbose=1,
                    callbacks=[es])
ypred = model.predict(XX)


fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.plot(history.history['loss'], label='train')
ax1.plot(history.history['val_loss'], label='test')
ax1.set_xlabel("epoch")
ax1.set_ylabel("loss")
ax1.legend()
ax1.set_title('loss of the neural model')
ax1.axis('tight')
#pyplot.show()

ax2.set_title('Training Set')
ax2.scatter(time, bikes, color="deepskyblue")
ax2.scatter(tt, ypred, color="lightgreen")
ax2.set_xlabel("time (days)")
ax2.set_ylabel("#bikes")
ax2.legend(["training data","predictions"],loc="upper right")
fig.autofmt_xdate()
scores = r2_score(yy,ypred)
print(scores)
# 0.969764426600568
plt.show()

C:\Users\rkhna\AppData\Local\Temp/ipykernel_3068/817614462.py:22: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


intial set :  17127 2016 3 12 9051
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 128)               3584      
_________________________________________________________________
dense_9 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 264       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 9         
Total params: 7,985
Trainable params: 7,985
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5000
73/73 [==============================] - 1s 4ms/step - loss: 37.3330 - val_loss: 18.4210
Epoch 2/5000
73/73 [==============================] - 0s 2ms/step - loss: 16.2256 - val_loss: 14.1316
Epoch 3/500

73/73 [==============================] - 0s 2ms/step - loss: 1.5747 - val_loss: 1.8234
Epoch 74/5000
73/73 [==============================] - 0s 2ms/step - loss: 1.4428 - val_loss: 1.6838
Epoch 75/5000
73/73 [==============================] - 0s 2ms/step - loss: 1.4839 - val_loss: 2.1624
Epoch 76/5000
73/73 [==============================] - 0s 2ms/step - loss: 1.6305 - val_loss: 2.9062
Epoch 77/5000
73/73 [==============================] - 0s 2ms/step - loss: 1.5152 - val_loss: 1.7533
Epoch 78/5000
73/73 [==============================] - 0s 2ms/step - loss: 1.3425 - val_loss: 2.0960
Epoch 79/5000
73/73 [==============================] - 0s 2ms/step - loss: 1.4104 - val_loss: 2.0079
Epoch 80/5000
73/73 [==============================] - 0s 2ms/step - loss: 1.5090 - val_loss: 1.7726
Epoch 81/5000
73/73 [==============================] - 0s 2ms/step - loss: 1.4136 - val_loss: 1.6937
Epoch 82/5000
73/73 [==============================] - 0s 2ms/step - loss: 1.3806 - val_loss: 1.5722
Epoc

73/73 [==============================] - 0s 2ms/step - loss: 0.8881 - val_loss: 1.3026
Epoch 154/5000
73/73 [==============================] - 0s 3ms/step - loss: 0.9877 - val_loss: 1.2475
Epoch 155/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.9295 - val_loss: 1.4331
Epoch 156/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.9430 - val_loss: 1.2275
Epoch 157/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.9062 - val_loss: 1.1609
Epoch 158/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.8885 - val_loss: 1.2706
Epoch 159/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.8200 - val_loss: 1.2357
Epoch 160/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.8932 - val_loss: 1.4116
Epoch 161/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.8813 - val_loss: 1.2625
Epoch 162/5000
73/73 [==============================] - 0s 2ms/step - loss: 1.0738 - val_loss: 1.

73/73 [==============================] - 0s 2ms/step - loss: 0.7598 - val_loss: 1.1374
Epoch 234/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.6780 - val_loss: 0.9541
Epoch 235/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.6216 - val_loss: 1.0644
Epoch 236/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.6924 - val_loss: 1.3675
Epoch 237/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.7676 - val_loss: 0.9849
Epoch 238/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.6567 - val_loss: 1.6535
Epoch 239/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.6828 - val_loss: 1.0308
Epoch 240/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.6535 - val_loss: 1.1163
Epoch 241/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.7726 - val_loss: 1.1745
Epoch 242/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.6305 - val_loss: 1.

...

73/73 [==============================] - 0s 2ms/step - loss: 0.2865 - val_loss: 0.6409
Epoch 1034/5000
73/73 [==============================] - 0s 3ms/step - loss: 0.2505 - val_loss: 0.6846
Epoch 1035/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2512 - val_loss: 0.6210
Epoch 1036/5000
73/73 [==============================] - 0s 3ms/step - loss: 0.2274 - val_loss: 0.6155
Epoch 1037/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2678 - val_loss: 0.6013
Epoch 1038/5000
73/73 [==============================] - 0s 3ms/step - loss: 0.2566 - val_loss: 0.5850
Epoch 1039/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2608 - val_loss: 0.7384
Epoch 1040/5000
73/73 [==============================] - 0s 3ms/step - loss: 0.2811 - val_loss: 0.5729
Epoch 1041/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2530 - val_loss: 0.6649
Epoch 1042/5000
73/73 [==============================] - 0s 3ms/step - loss: 0.2797 - val

73/73 [==============================] - 0s 2ms/step - loss: 0.2544 - val_loss: 0.6227
Epoch 1113/5000
73/73 [==============================] - 0s 3ms/step - loss: 0.2495 - val_loss: 0.6756
Epoch 1114/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2503 - val_loss: 0.5995
Epoch 1115/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2961 - val_loss: 0.6486
Epoch 1116/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2777 - val_loss: 0.7214
Epoch 1117/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2494 - val_loss: 0.5794
Epoch 1118/5000
73/73 [==============================] - 0s 3ms/step - loss: 0.2544 - val_loss: 0.5577
Epoch 1119/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2505 - val_loss: 0.5323
Epoch 1120/5000
73/73 [==============================] - 0s 3ms/step - loss: 0.2773 - val_loss: 0.6030
Epoch 1121/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2521 - val

73/73 [==============================] - 0s 2ms/step - loss: 0.2782 - val_loss: 0.8014
Epoch 1192/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2519 - val_loss: 0.6787
Epoch 1193/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2401 - val_loss: 0.5550
Epoch 1194/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2789 - val_loss: 0.6177
Epoch 1195/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2407 - val_loss: 0.6251
Epoch 1196/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2337 - val_loss: 0.5589
Epoch 1197/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2369 - val_loss: 0.5885
Epoch 1198/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2298 - val_loss: 0.6743
Epoch 1199/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2329 - val_loss: 0.5012
Epoch 1200/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2522 - val

73/73 [==============================] - 0s 2ms/step - loss: 0.2235 - val_loss: 0.6480
Epoch 1271/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2260 - val_loss: 0.5425
Epoch 1272/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2393 - val_loss: 0.6040
Epoch 1273/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2182 - val_loss: 0.5698
Epoch 1274/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2610 - val_loss: 0.5785
Epoch 1275/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2545 - val_loss: 0.5359
Epoch 1276/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2324 - val_loss: 0.5840
Epoch 1277/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2501 - val_loss: 0.6266
Epoch 1278/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2433 - val_loss: 0.5610
Epoch 1279/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2293 - val

73/73 [==============================] - 0s 2ms/step - loss: 0.2106 - val_loss: 0.5504
Epoch 1350/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2345 - val_loss: 0.5340
Epoch 1351/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2091 - val_loss: 0.5959
Epoch 1352/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2095 - val_loss: 0.5502
Epoch 1353/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2006 - val_loss: 0.5027
Epoch 1354/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2180 - val_loss: 0.5516
Epoch 1355/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2405 - val_loss: 0.6095
Epoch 1356/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2208 - val_loss: 0.5104
Epoch 1357/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.2514 - val_loss: 0.7669
Epoch 1358/5000
73/73 [==============================] - 0s 2ms/step - loss: 0.3672 - val

In [8]:
# identify depth for Random Forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import math
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from matplotlib import pyplot
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score

%matplotlib qt 

t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]


q=2
lag=3
stride=1
w=math.floor(7*24*60*60/dt)
length = bikes.size - w - lag * w - q
print("intial set : ", bikes.size, w, lag, q, length)
XX=bikes[q:q+length:stride]
X1 = day_of_month[q:q+length:stride]
X2 = hour[q:q+length:stride]
XX=np.column_stack((XX,X1, X2))
#week
for i in range(1,lag):
    X=bikes[i*w+q:i*w+q+length:stride]
    X1 = day_of_month[i*w+q:i*w+q+length:stride]
    X2 = hour[i*w+q:i*w+q+length:stride]
    XX=np.column_stack((XX,X, X1, X2))
d=math.floor(24*60*60/dt)
#days
for i in range(0,lag):
    X=bikes[i*d+q:i*d+q+length:stride]
    X1=day_of_month[i*d+q:i*d+q+length:stride]
    X2=hour[i*d+q:i*d+q+length:stride]
    XX=np.column_stack((XX,X, X1, X2))

for i in range(0,lag):
    X=bikes[i:i+length:stride]
    X1=day_of_month[i:i+length:stride]
    X2=hour[i:i+length:stride]
    XX=np.column_stack((XX,X, X1, X2))

yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
tt=time[lag*w+w+q:lag*w+w+q+length:stride]

yy.reset_index(drop=True, inplace=True)
tt.reset_index(drop=True, inplace=True)


train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

regr = RandomForestRegressor(max_depth=None).fit(XX[train], yy[train])
y_pred = regr.predict(XX)
score = regr.score(XX[test], yy[test])
print(score)

dummy = DummyRegressor(strategy="mean").fit(XX[train], yy[train])
print("Dummy Mean Score")
print(dummy.score(XX[test], yy[test]))

mean_accuracy = []
std_dev = []

depth = [None, 5, 10, 50, 100, 500, 1000]
depth_str = ["None", "5", "10", "50", "100", "500", "1000"]

for each_depth in depth:
    model = RandomForestRegressor(max_depth=each_depth).fit(XX[train], yy[train])
    print(f'{model.score(XX[test], yy[test])} - {each_depth}')
    cv_scores = cross_val_score(regr, XX[test], yy[test], cv=5, scoring='neg_root_mean_squared_error')
    mean_accuracy.append(np.array(cv_scores).mean())
    std_dev.append(np.array(cv_scores).std())

plt.close("all")
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.ticklabel_format(style='plain')
ax1.errorbar(depth_str,mean_accuracy,yerr=std_dev,linewidth=3)
ax1.set_xlabel('Depth for random Forest')
ax1.set_ylabel('Root Mean Squared Error')
ax1.set_title('Cross Validation RMSE scores')

actual_data_handler = ax2.scatter(time, bikes, marker="o", color="Green", alpha=0.3, label = "training data")
pred_data_handler = ax2.scatter(x=tt, y=y_pred, marker="+", color="Red", alpha=0.3, label = "predictions")
ax2.title.set_text('training/prediction plot')
ax2.legend(loc="upper right")
ax2.set_xlabel("Time of Day")
ax2.set_ylabel("Cycle Count")
fig.autofmt_xdate()
plt.show()

C:\Users\rkhna\AppData\Local\Temp/ipykernel_14984/631550111.py:17: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000


intial set :  17127 2016 3 2 9061
0.9953807262180562
Dummy Mean Score
-0.0010733029707570285
0.9954109689181536 - None
0.8624508244374849 - 5
0.9827169912951408 - 10
0.995151821371214 - 50
0.9950784784485558 - 100
0.9953730927512536 - 500
0.9949134487233444 - 1000


In [16]:
# CNN for all q

from sklearn.metrics import mean_squared_error
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import numpy
from tensorflow.keras.optimizers import Adam
import keras
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

%matplotlib qt 

t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
dt = t_full[1]-t_full[0]

day_of_month = suburb_df['day_of_month']
hour = suburb_df['hour']

qrange = [2, 6, 12]
lag=3
stride=1
plt.close("all")
fig, ax = plt.subplots(3, 2)
ax_index = 0
q_score = {}
for q in qrange:
    w=math.floor(7*24*60*60/dt)
    length = bikes.size - w - lag * w - q
    XX=bikes[q:q+length:stride]
    X1 = day_of_month[q:q+length:stride]
    X2 = hour[q:q+length:stride]
    XX=np.column_stack((XX,X1, X2))
    #week
    for i in range(1,lag):
        X=bikes[i*w+q:i*w+q+length:stride]
        X1 = day_of_month[i*w+q:i*w+q+length:stride]
        X2 = hour[i*w+q:i*w+q+length:stride]
        XX=np.column_stack((XX,X, X1, X2))
    d=math.floor(24*60*60/dt)
    #days
    for i in range(0,lag):
        X=bikes[i*d+q:i*d+q+length:stride]
        X1=day_of_month[i*d+q:i*d+q+length:stride]
        X2=hour[i*d+q:i*d+q+length:stride]
        XX=np.column_stack((XX,X, X1, X2))

    for i in range(0,lag):
        X=bikes[i:i+length:stride]
        X1=day_of_month[i:i+length:stride]
        X2=hour[i:i+length:stride]
        XX=np.column_stack((XX,X, X1, X2))

    yy=bikes[lag*w+w+q:lag*w+w+q+length:stride] 
    tt=time[lag*w+w+q:lag*w+w+q+length:stride]

    yy.reset_index(drop=True, inplace=True)
    tt.reset_index(drop=True, inplace=True)


    train, test = train_test_split(np.arange(0,yy.size),test_size=0.2)

    model = Sequential()
    model.add(Dense(128, activation="relu", input_dim=27))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(8, activation="relu"))
    model.add(Dense(1, activation="linear"))
    model.compile(loss='mean_squared_error', optimizer=Adam(lr=1e-3, decay=1e-3 / 200))
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
    # 10000000
    # 15000
    # verbose = 2
    history = model.fit(XX[train], yy[train], validation_data=(XX[test], yy[test]), epochs=5000, batch_size=100, verbose=0,
                        callbacks=[es])
    ypred = model.predict(XX)

    print("index --- ", i)
    
    ax[ax_index, 0].set_title(f'loss of the neural model for {q}')
    ax[ax_index, 0].plot(history.history['loss'], label='train')
    ax[ax_index, 0].plot(history.history['val_loss'], label='test')
    ax[ax_index, 0].set_xlabel("epoch")
    ax[ax_index, 0].set_ylabel("loss")
    ax[ax_index, 0].legend()
    ax[ax_index, 0].axis('tight')
    #pyplot.show()

    ax[ax_index, 1].set_title(f'Training Set for {q}')
    ax[ax_index, 1].scatter(time, bikes, color="deepskyblue", marker = '.')
    ax[ax_index, 1].scatter(tt, ypred, color="lightgreen", marker = '.')
    ax[ax_index, 1].set_xlabel("time (days)")
    ax[ax_index, 1].set_ylabel("#bikes")
    ax[ax_index, 1].legend(["training data","predictions"],loc="upper right")
    # fig.autofmt_xdate()
    scores = r2_score(yy,ypred)
    print(q, scores)
    q_score[q] = scores
    ax_index += 1
# 0.969764426600568

for q, score in q_score.items():
    print(f"q value {q} - score {score}")

fig.autofmt_xdate()
plt.show()

C:\Users\rkhna\AppData\Local\Temp/ipykernel_3068/2769184219.py:24: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  t_full = pd.array(pd.DatetimeIndex(suburb_df.iloc[:,1]).astype(np.int64))/1000000000
C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 02086: early stopping
index ---  2
2 0.9925863971871264


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 01204: early stopping
index ---  2
6 0.9949443288052098


C:\Users\rkhna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 01456: early stopping
index ---  2
12 0.9939332916249415
q value 2 - score 0.9925863971871264
q value 6 - score 0.9949443288052098
q value 12 - score 0.9939332916249415


In [12]:
# BASELINE MODEL

# https://machinelearningmastery.com/seasonal-persistence-forecasting-python/

forecast = pd.DataFrame(bikes.values)

d=math.floor(24*60*60/dt)
q=12
dummy_value = pd.concat([forecast.shift(d+q), forecast], axis=1)
dummy_value.columns = ['t,', 't+1']

X = dummy_value.values
train_size = int(len(X) * 0.66)
train, test = X[1:train_size], X[train_size:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]


def model_persistence(x):
    return x

predictions = list()
for x in test_X:
    yhat = model_persistence(x)
    predictions.append(yhat)
mse = mean_squared_error(test_y, predictions)
rmse = math.sqrt(mse)
print('Test MSE: %.3f' % mse)
print('Test RMSE: %.3f' % rmse)
print(r2_score(test_y, predictions))

Test MSE: 29.655
Test RMSE: 5.446
0.07251138070997698


In [171]:
dummy_value

,"t-1,",t+1
0,NaN,0
1,NaN,0
2,NaN,0
3,NaN,0
4,NaN,0
...,...,...
17122,6.0,2
17123,6.0,2
17124,6.0,2
17125,6.0,2
